## dummy SUAVE Vehicle( only Fuselage)

In [1]:
import SUAVE
from SUAVE.Core import Units, Data
import vsp
import numpy as np

vehicle = SUAVE.Vehicle()
vehicle.tag = 'test'    

# ------------------------------------------------------------------
#  Fuselage
# ------------------------------------------------------------------

c = SUAVE.Components.Fuselages.Fuselage()
fuselage.tag = 'fuselage'

fuselage.number_coach_seats    = vehicle.passengers
fuselage.seats_abreast         = 6
fuselage.seat_pitch            = 1     * Units.meter
fuselage.fineness.nose         = 1.6
fuselage.fineness.tail         = 2.
fuselage.lengths.nose          = 6.4   * Units.meter
fuselage.lengths.tail          = 8.0   * Units.meter
fuselage.lengths.cabin         = 28.85 * Units.meter
fuselage.lengths.total         = 38.02 * Units.meter
fuselage.lengths.fore_space    = 6.    * Units.meter
fuselage.lengths.aft_space     = 5.    * Units.meter
fuselage.width                 = 3.74  * Units.meter
fuselage.heights.maximum       = 3.74  * Units.meter
fuselage.effective_diameter    = 3.74     * Units.meter
fuselage.areas.side_projected  = 142.1948 * Units['meters**2'] 
fuselage.areas.wetted          = 446.718  * Units['meters**2'] 
fuselage.areas.front_projected = 12.57    * Units['meters**2'] 
fuselage.differential_pressure = 5.0e4 * Units.pascal # Maximum differential pressure

fuselage.heights.at_quarter_length          = 3.74 * Units.meter
fuselage.heights.at_three_quarters_length   = 3.65 * Units.meter
fuselage.heights.at_wing_root_quarter_chord = 3.74 * Units.meter

# add to vehicle
vehicle.append_component(fuselage)


In [5]:
fuselage.append("Xsec")

AttributeError: 'str' object has no attribute 'tag'

## SUAVE vsp write(only Fuselage)

In [2]:
# -------------
# Fuselage
# -------------    

fuselage = vehicle.fuselages.fuselage
width    = fuselage.width
length   = fuselage.lengths.total
hmax     = fuselage.heights.maximum
height1  = fuselage.heights.at_quarter_length
height2  = fuselage.heights.at_wing_root_quarter_chord 
height3  = fuselage.heights.at_three_quarters_length
effdia   = fuselage.effective_diameter
n_fine   = fuselage.fineness.nose 
t_fine   = fuselage.fineness.tail  
# w_ac     = wing.aerodynamic_center

# w_origin = vehicle.wings.main_wing.origin
# w_c_4    = vehicle.wings.main_wing.chords.root/4.

In [3]:
# Figure out the location x location of each section, 3 sections, end of nose, wing origin, and start of tail

x1 = n_fine*width/length
# x2 = (w_origin[0]+w_c_4)/length
# x3 = 1-t_fine*width/length

x2 = 2
x3 = 3

fuse_id = vsp.AddGeom("FUSELAGE") 
vsp.SetGeomName(fuse_id, fuselage.tag)
# area_tags[fuselage.tag] = ['fuselages',fuselage.tag]

tail_z_pos = 0.02 # default value

In [4]:
if fuselage.has_key('OpenVSP_values'):
    vals = fuselage.OpenVSP_values

    # for wave drag testing
    fuselage.OpenVSP_ID = fuse_id

    # Nose
    vsp.SetParmVal(fuse_id,"TopLAngle","XSec_0",vals.nose.top.angle)
    vsp.SetParmVal(fuse_id,"TopLStrength","XSec_0",vals.nose.top.strength)
    vsp.SetParmVal(fuse_id,"RightLAngle","XSec_0",vals.nose.side.angle)
    vsp.SetParmVal(fuse_id,"RightLStrength","XSec_0",vals.nose.side.strength)
    vsp.SetParmVal(fuse_id,"TBSym","XSec_0",vals.nose.TB_Sym)
    vsp.SetParmVal(fuse_id,"ZLocPercent","XSec_0",vals.nose.z_pos)


    # Tail
    vsp.SetParmVal(fuse_id,"TopLAngle","XSec_4",vals.tail.top.angle)
    vsp.SetParmVal(fuse_id,"TopLStrength","XSec_4",vals.tail.top.strength)
    # Below can be enabled if AllSym (below) is removed
    #vsp.SetParmVal(fuse_id,"RightLAngle","XSec_4",vals.tail.side.angle)
    #vsp.SetParmVal(fuse_id,"RightLStrength","XSec_4",vals.tail.side.strength)
    #vsp.SetParmVal(fuse_id,"TBSym","XSec_4",vals.tail.TB_Sym)
    #vsp.SetParmVal(fuse_id,"BottomLAngle","XSec_4",vals.tail.bottom.angle)
    #vsp.SetParmVal(fuse_id,"BottomLStrength","XSec_4",vals.tail.bottom.strength)
    if vals.tail.has_key('z_pos'):
        tail_z_pos = vals.tail.z_pos
    else:
        pass # use above default

    vsp.SetParmVal(fuse_id,"AllSym","XSec_4",1)

vsp.SetParmVal(fuse_id,"Length","Design",length)
vsp.SetParmVal(fuse_id,"Diameter","Design",width)
vsp.SetParmVal(fuse_id,"XLocPercent","XSec_1",x1)
vsp.SetParmVal(fuse_id,"XLocPercent","XSec_2",x2)
vsp.SetParmVal(fuse_id,"XLocPercent","XSec_3",x3)
vsp.SetParmVal(fuse_id,"ZLocPercent","XSec_4",tail_z_pos)
vsp.SetParmVal(fuse_id, "Ellipse_Width", "XSecCurve_1", width)
vsp.SetParmVal(fuse_id, "Ellipse_Width", "XSecCurve_2", width)
vsp.SetParmVal(fuse_id, "Ellipse_Width", "XSecCurve_3", width)
vsp.SetParmVal(fuse_id, "Ellipse_Height", "XSecCurve_1", height1);
vsp.SetParmVal(fuse_id, "Ellipse_Height", "XSecCurve_2", height2);
vsp.SetParmVal(fuse_id, "Ellipse_Height", "XSecCurve_3", height3);   

# Write the vehicle to the file

vsp.WriteVSPFile(fuselage.tag + ".vsp3")

In [16]:
! explorer .